efetua os imports das dependencias

In [0]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

importa os dados do arquivo excel referente ao civil e adiciona a coluna CLASSE com o valor 0, representando o tipo de dado civil

In [0]:
uri_data_civel = 'https://github.com/julianopacheco/classificacao-textos-juridicos/blob/master/arquivos/xls/civel.xlsx?raw=true'
dados_civil = pd.read_excel(uri_data_civel)
dados_civil["CLASSE"] = 0

importa os dados do arquivo excel referente ao crime e adiciona a coluna CLASSE com o valor 1, representando o tipo de dado crime

In [0]:
uri_data_crime = 'https://github.com/julianopacheco/classificacao-textos-juridicos/blob/master/arquivos/xls/crime.xlsx?raw=true'
dados_crime = pd.read_excel(uri_data_crime)
dados_crime["CLASSE"] = 1

concatena as duas fontes de dados e gera uma nova contendo as duas

In [0]:
dados_civil_e_crime = pd.concat([dados_civil, dados_crime], axis=0)

como agora a fonte de dados contem as duas listas e necessário embaralhar elas para que o civil e crime nao venha na ordem que foram concatenados

isso e feito com o shuffle

In [0]:
dados_civil_e_crime = shuffle(dados_civil_e_crime)
dados_civil_e_crime.head()

,PROCESSO,EMENTA,DATA,CLASSE
3815,70077666493,APELAÇÃO CÍVEL. AÇÕES REVISIONAL E DE BUSCA E ...,28/06/18,0
1217,70075943191,APELAÇÃO CÍVEL. RESPONSABILIDADE CIVIL. RESPON...,28/02/18,0
172,70072029234,HABEAS CORPUS. CRIMES DE ROUBO DUPLAMENTE MAJO...,22/02/17,1
305,70075807909,APELAÇÃO CÍVEL. ACIDENTE DE TRABALHO. FORMA DE...,13/12/17,0
5256,70078829223,APELAÇÃO. MANDATO. AÇÃO INDENIZATÓRIA. PRELIMI...,13/09/18,0


verifica a proporcionalidade de cada um dos tipos (civil = 0, crime = 1)

In [0]:
dados_civil_e_crime.CLASSE.value_counts()

1    6000
0    6000
Name: CLASSE, dtype: int64

extrai as ementas dos dados

In [0]:
ementas = dados_civil_e_crime["EMENTA"]


instancia o CountVectorizer e TfidfVectorizer para vetorizar os textos das ementas

In [0]:
count_vectorizer = CountVectorizer(max_features = 50)
tfidf_vetorizar = TfidfVectorizer(max_features = 50)
tfidf_vetorizar_use_idf = TfidfVectorizer(max_features = 50, use_idf=False)

cria os bag of words através da transformação com os vetores CountVectorizer e TfidfVectorizer

In [0]:
bag_of_words_count = count_vectorizer.fit_transform(ementas)
bag_of_words_tfidf = tfidf_vetorizar.fit_transform(ementas)
bag_of_words_tfidf_use_idf = tfidf_vetorizar_use_idf.fit_transform(ementas)

gera as variaveis treino, teste, classe_treino, classe_teste através do train_test_split passando um seed de 48

estas variaveis serão utilizadas abaixo na aplicação dos algortimos

In [0]:

treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words_count,
                                                              dados_civil_e_crime.CLASSE,
                                                              random_state = 48)
treino_tfidf, teste_tfidf, classe_treino_tfidf, classe_teste_tfidf = train_test_split(bag_of_words_tfidf,
                                                                                      dados_civil_e_crime.CLASSE,
                                                                                      random_state = 48)
treino_tfidf_use_idf, teste_tfidf_use_idf, classe_treino_tfidf_use_idf, classe_teste_tfidf_use_idf = train_test_split(bag_of_words_tfidf_use_idf,
                                                                                                                      dados_civil_e_crime.CLASSE,
                                                                                                                      random_state = 48)

define metodos abaixo para testar a regressão logistica e o svc

define o metodo que aplica e escreve os resultados do algortimo LogisticRegression

In [0]:
def execute_LogisticRegression(treino, classe_treino, teste, classe_teste):
  regressao_logistica = LogisticRegression(solver='lbfgs', max_iter=10000)
  regressao_logistica.fit(treino, classe_treino)
  #acuracia = regressao_logistica.score(teste, classe_teste)
  #print(f'LogisticRegression acuracia: {acuracia}')
  predicted = regressao_logistica.predict(teste)
  print(classification_report(classe_teste, predicted))

define um método que cria um pipeline para normalizar os dados e instanciar o algortimo SVC

efetua o treino, aplica o metodo que irá prever o teste e escreve o relatório da aplicação

In [0]:
def execute_SVC(treino, classe_treino, teste, classe_teste):
  pipeline = Pipeline([
    ('normalizer', Normalizer()),
    ('svc', SVC(gamma='auto'))
  ])

  pipeline.fit(treino, classe_treino)
  predicted = pipeline.predict(teste)
  print(classification_report(classe_teste, predicted))

executa o algortimo LogisticRegression passando os dados referentes ao CountVectorizer

In [0]:
execute_LogisticRegression(treino, classe_treino, teste, classe_teste)

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      1499
           1       0.98      0.97      0.97      1501

   micro avg       0.97      0.97      0.97      3000
   macro avg       0.97      0.97      0.97      3000
weighted avg       0.97      0.97      0.97      3000



executa o algortimo LogisticRegression passando os dados referentes ao TfidfVectorizer

In [0]:
execute_LogisticRegression(treino_tfidf, classe_treino_tfidf, teste_tfidf, classe_teste_tfidf)

              precision    recall  f1-score   support

           0       0.95      0.98      0.96      1499
           1       0.98      0.95      0.96      1501

   micro avg       0.96      0.96      0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000



executa o algortimo SVC passando os dados referentes ao TfidfVectorizer(use_idf)

In [0]:
execute_LogisticRegression(treino_tfidf_use_idf, classe_treino_tfidf_use_idf, teste_tfidf_use_idf, classe_teste_tfidf_use_idf)

              precision    recall  f1-score   support

           0       0.94      0.98      0.96      1499
           1       0.98      0.94      0.96      1501

   micro avg       0.96      0.96      0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000



executa o algortimo SVC passando os dados referentes ao CountVectorizer

In [0]:
execute_SVC(treino, classe_treino, teste, classe_teste)

              precision    recall  f1-score   support

           0       0.89      0.98      0.94      1499
           1       0.98      0.88      0.93      1501

   micro avg       0.93      0.93      0.93      3000
   macro avg       0.94      0.93      0.93      3000
weighted avg       0.94      0.93      0.93      3000



executa o algortimo SVC passando os dados referentes ao TfidfVectorizer

In [0]:
execute_SVC(treino_tfidf, classe_treino_tfidf, teste_tfidf, classe_teste_tfidf)

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      1499
           1       0.98      0.90      0.94      1501

   micro avg       0.94      0.94      0.94      3000
   macro avg       0.95      0.94      0.94      3000
weighted avg       0.95      0.94      0.94      3000



executa o algortimo SVC passando os dados referentes ao TfidfVectorizer(use_idf)

In [0]:
execute_SVC(treino_tfidf_use_idf, classe_treino_tfidf_use_idf, teste_tfidf_use_idf, classe_teste_tfidf_use_idf)

              precision    recall  f1-score   support

           0       0.89      0.98      0.94      1499
           1       0.98      0.88      0.93      1501

   micro avg       0.93      0.93      0.93      3000
   macro avg       0.94      0.93      0.93      3000
weighted avg       0.94      0.93      0.93      3000

